
# <div align="center">  The Effect of Various Factors on Income


Contributors: Mohammed Ahmed, Kevin Wu, Fei Yuan, Mazin Karjikar


## <div align="center"> Introduction

Welcome to our data science tutorial! In this tutorial we will be showcasing the data science pipeline. To do that, we have chosen to analyze an interesting topic - factors that affect income! We have 

In [ ]:
# Import page, not all of them are useful atm
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

Process crime dataset

In [ ]:
raw1 = pd.DataFrame(columns = ['Metropolitan Statistical Area', 'Counties/principal cities',
       'Population', 'Violent crime', 'Murder and nonnegligent manslaughter',
       'Rape', 'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny-theft', 'Motor vehicle theft', 'Year'])
for i in np.arange(2005,2020):
  if i == 2016: link = 'https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/topic-pages/tables/table-4'
  elif i > 2016: link = 'https://ucr.fbi.gov/crime-in-the-u.s/'+str(i)+'/crime-in-the-u.s.-'+str(i)+'/topic-pages/tables/table-6'
  elif (i == 2012 or i == 2013): 
    link = 'https://ucr.fbi.gov/crime-in-the-u.s/'+str(i)+'/crime-in-the-u.s.-'+str(i)+'/tables/6tabledatadecpdf/table-6'
  elif i >= 2010: link = 'https://ucr.fbi.gov/crime-in-the-u.s/'+str(i)+'/crime-in-the-u.s.-'+str(i)+'/tables/table-6'
  elif i > 2005: link = 'https://www2.fbi.gov/ucr/cius'+str(i)+'/data/table_06.html'
  elif i == 2005: link = 'https://www2.fbi.gov/ucr/05cius/data/table_06.html'
  req = requests.get(link)
  soup = BeautifulSoup(req.content, 'html.parser')
  if i > 2009: id = 'table-data-container'
  else: id = 'datatable'
  snip = soup.find(id = id).prettify
  temp = pd.read_html(str(snip))[0]
  temp['Year'] = i
  temp.columns = ['Metropolitan Statistical Area', 'Counties/principal cities',
       'Population', 'Violent crime', 'Murder and nonnegligent manslaughter',
       'Rape', 'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny-theft', 'Motor vehicle theft', 'Year']
  raw1 = pd.concat([raw1, temp])
  print(str(i) + ' request successful')
raw1

2005 request successful
2006 request successful
2007 request successful
2008 request successful
2009 request successful
2010 request successful
2011 request successful
2012 request successful
2013 request successful
2014 request successful
2015 request successful
2016 request successful
2017 request successful
2018 request successful
2019 request successful


,Metropolitan Statistical Area,Counties/principal cities,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Year
0,"Abilene, TX M.S.A.","Callahan, Jones, and Taylor",161123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005
1,"Abilene, TX M.S.A.",City of Abilene,116695,576.0,5.0,78.0,146.0,347.0,5504.0,1521.0,3685.0,298.0,2005
2,"Abilene, TX M.S.A.",Total area actually reporting,100.0%,640.0,7.0,82.0,156.0,395.0,6196.0,1769.0,4076.0,351.0,2005
3,"Abilene, TX M.S.A.","Rate per 100,000 inhabitants",NaN,397.2,4.3,50.9,96.8,245.2,3845.5,1097.9,2529.7,217.8,2005
4,"Akron, OH M.S.A.",Portage and Summit,702386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,"San Juan-Bayamon-Caguas, Puerto Rico M.S.A.","Rate per 100,000 inhabitants",NaN,227.3,23.6,6.8,86.7,110.2,845.0,143.1,542.0,159.9,2019
1929,"Yauco, Puerto Rico M.S.A.",NaN,85830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1930,"Yauco, Puerto Rico M.S.A.","Includes Guanica, Guayanilla, Penuelas, and Ya...",85830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1931,"Yauco, Puerto Rico M.S.A.",Total area actually reporting,100.0%,148.0,10.0,11.0,19.0,108.0,267.0,66.0,184.0,17.0,2019


In [ ]:
# Refining raw data
crates = raw1.copy().loc[raw1['Counties/principal cities'] == 'Rate per 100,000 inhabitants',]
crates = crates.drop(['Population', 'Counties/principal cities'], axis = 1)
crates['Metropolitan Statistical Area'] = crates['Metropolitan Statistical Area'].replace(r' (\S\.)*(\S)?(\d)?(,( )?\d)*?$', '', regex = True)
crates

,Metropolitan Statistical Area,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Year
3,"Abilene, TX",397.2,4.3,50.9,96.8,245.2,3845.5,1097.9,2529.7,217.8,2005
8,"Akron, OH",266.4,5.4,47.3,116.6,97.1,3662.1,854.5,2504.2,303.4,2005
13,"Albany, GA",374.8,4.8,30.5,139.3,200.3,4057.2,1306.8,2502.3,248.1,2005
19,"Albany-Schenectady-Troy, NY",365.7,3.0,26.6,107.0,229.2,2662.1,532.9,1972.0,157.1,2005
24,"Albuquerque, NM",838.9,9.3,47.6,168.9,613.1,4914.4,1097.9,3169.4,647.0,2005
...,...,...,...,...,...,...,...,...,...,...,...
1916,"Mayaguez, Puerto Rico",112.7,4.2,7.4,26.3,74.8,398.0,74.8,292.7,30.5,2019
1920,"Ponce, Puerto Rico",165.4,12.1,3.7,62.2,87.3,573.6,112.4,383.2,78.0,2019
1924,"San German, Puerto Rico",83.2,10.7,2.5,5.8,64.2,331.8,78.2,238.8,14.8,2019
1928,"San Juan-Bayamon-Caguas, Puerto Rico",227.3,23.6,6.8,86.7,110.2,845.0,143.1,542.0,159.9,2019


Process air quality dataset 

In [ ]:
raw2 = pd.read_excel('AirQuality.xlsx')
raw2.columns = pd.concat([raw2.iloc[2,:5], raw2.iloc[2,5:].astype(int)])
raw2 = raw2.iloc[3:858,:]
raw2.index = np.arange(1,len(raw2)+1)
temp = raw2.loc[1,'Core Based Statistical Area']
for i, j in zip(raw2['Core Based Statistical Area'], raw2.index):
  if not pd.isnull(i): temp = i
  else: raw2.loc[j,'Core Based Statistical Area'] = temp
raw2

2,CBSA,Core Based Statistical Area,Pollutant,Trend Statistic,Number of Trend Sites,1990,1991,1992,1993,1994,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,10100,"Aberdeen, SD",PM2.5,Weighted Annual Mean,1,NaN,NaN,NaN,NaN,NaN,...,7.500,7.300,6.200,6.200,5.400,5.900,6.700,5.900,5.900,5.900
2,10100,"Aberdeen, SD",PM2.5,98th Percentile,1,NaN,NaN,NaN,NaN,NaN,...,23.000,22.000,17.000,14.000,14.000,13.000,22.000,18.000,18.000,18.000
3,10420,"Akron, OH",O3,4th Max,2,0.090,0.101,0.087,0.093,0.086,...,0.072,0.059,0.060,0.065,0.060,0.066,0.068,0.062,0.063,0.067
4,10420,"Akron, OH",PM2.5,Weighted Annual Mean,3,NaN,NaN,NaN,NaN,NaN,...,10.000,9.700,9.900,10.400,8.200,7.900,7.900,8.200,8.000,8.300
5,10420,"Akron, OH",PM2.5,98th Percentile,3,NaN,NaN,NaN,NaN,NaN,...,19.000,24.000,22.000,23.000,17.000,18.000,18.000,21.000,21.000,20.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,49700,"Yuba City, CA",NO2,Annual Mean,1,17.000,17.000,17.000,17.000,16.000,...,10.000,10.000,8.000,7.000,7.000,7.000,7.000,6.000,6.000,5.000
852,49700,"Yuba City, CA",NO2,98th Percentile,1,70.000,70.000,70.000,70.000,65.000,...,46.000,52.000,44.000,39.000,40.000,42.000,41.000,40.000,38.000,34.000
853,49700,"Yuba City, CA",O3,4th Max,1,0.076,0.080,0.090,0.078,0.089,...,0.069,0.060,0.069,0.064,0.063,0.067,0.065,0.061,0.066,0.072
854,49700,"Yuba City, CA",PM2.5,Weighted Annual Mean,1,NaN,NaN,NaN,NaN,NaN,...,6.900,8.200,9.400,9.600,8.100,9.300,10.300,8.400,16.400,14.500


In [ ]:
# Tidy data + refining & trimming
temp2 = pd.melt(raw2, id_vars = ['CBSA', 'Core Based Statistical Area', 'Pollutant', 'Trend Statistic', 'Number of Trend Sites'],
                      value_vars = raw2.columns[5:], var_name = 'Year')
temp2 = temp2.loc[((temp2.Pollutant == 'PM2.5') | (temp2.Pollutant == 'O3')) & ((temp2['Trend Statistic'] == 'Weighted Annual Mean') 
                  | (temp2['Trend Statistic'] == '4th Max'))].drop('Trend Statistic', axis = 1)

temp2 = pd.pivot_table(temp2, values = 'value', index = [temp2.index, 'CBSA', 'Core Based Statistical Area', 'Year'], columns = 'Pollutant', aggfunc=np.sum)
temp2 = temp2.reset_index().drop('level_0', axis = 1)
temp2.columns = ['CBSA', 'Core Based Statistical Area', 'Year', 'O3 4th Maximum', 'PM2.5 Weighted Annual Mean']
aggfun = {'CBSA':'first', 'PM2.5 Weighted Annual Mean':'sum', 'O3 4th Maximum':'sum'}
temp2 = temp2.sort_values(by = ['Core Based Statistical Area', 'Year']).groupby([temp2['Core Based Statistical Area'], temp2.Year]).aggregate(aggfun)
temp2 = temp2.reset_index()
temp2.columns = ['Metropolitan Statistical Area', 'Year', 'CBSA', 'PM2.5 Weighted Annual Mean', 'O3 4th Maximum']
temp2['O3 4th Maximum'].replace(0,np.nan,inplace=True)
temp2['PM2.5 Weighted Annual Mean'].replace(0,np.nan,inplace=True)
temp2 = temp2[['Metropolitan Statistical Area', 'CBSA', 'Year', 'PM2.5 Weighted Annual Mean', 'O3 4th Maximum']]

airQuality = temp2.loc[temp2.Year >= 2005,:]
airQuality

,Metropolitan Statistical Area,CBSA,Year,PM2.5 Weighted Annual Mean,O3 4th Maximum
15,"Aberdeen, SD",10100,2005,9.0,NaN
16,"Aberdeen, SD",10100,2006,8.2,NaN
17,"Aberdeen, SD",10100,2007,8.0,NaN
18,"Aberdeen, SD",10100,2008,7.7,NaN
19,"Aberdeen, SD",10100,2009,8.1,NaN
...,...,...,...,...,...
8923,"Yuba City, CA",49700,2017,9.3,0.067
8924,"Yuba City, CA",49700,2018,10.3,0.065
8925,"Yuba City, CA",49700,2019,8.4,0.061
8926,"Yuba City, CA",49700,2020,16.4,0.066


Current merged dataset

In [ ]:
# Current collective dataset
data = pd.merge(airQuality, crates, how = 'inner', on = ['Metropolitan Statistical Area', 'Year'])
data.index = np.arange(1,len(data)+1)
data

,Metropolitan Statistical Area,CBSA,Year,PM2.5 Weighted Annual Mean,O3 4th Maximum,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft
1,"Akron, OH",10420,2005,15.7,0.091,266.4,5.4,47.3,116.6,97.1,3662.1,854.5,2504.2,303.4
2,"Akron, OH",10420,2007,14.1,0.087,313.1,3.6,44.2,136.5,128.9,3357.4,825.9,2265.9,265.7
3,"Akron, OH",10420,2008,12.9,0.075,347.2,3.4,40.3,141.0,162.4,3312.3,882.7,2213.0,216.6
4,"Akron, OH",10420,2009,11.7,0.067,357.1,4.0,44.4,130.7,178.0,3268.4,869.6,2198.2,200.6
5,"Akron, OH",10420,2010,12.4,0.074,304.7,3.7,40.9,105.1,155.0,3185.6,947.7,2074.5,163.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,"Yuba City, CA",49700,2015,9.6,0.064,371.6,3.5,37.0,81.6,249.5,2588.6,659.3,1555.9,373.4
2598,"Yuba City, CA",49700,2016,8.1,0.063,397.5,7.6,29.3,80.3,280.3,2717.6,643.2,1641.7,432.7
2599,"Yuba City, CA",49700,2017,9.3,0.067,386.6,2.3,31.9,84.6,267.8,2850.9,672.3,1614.1,564.5
2600,"Yuba City, CA",49700,2018,10.3,0.065,386.0,4.6,47.3,86.0,248.1,2510.7,483.6,1585.7,441.4
